In [1]:
from datetime import datetime, timedelta
from pathlib import Path

import polars as pl
from sklearn.datasets import make_regression

from timelake import TimeLake
from timelake.constants import StorageType

In [2]:
# Params
n_hours = 24 * 30  # 30 days
n_features = 100

# Generate regression features
X, _ = make_regression(n_samples=n_hours, n_features=n_features)
timestamps = [datetime(2024, 1, 1) + timedelta(hours=i) for i in range(n_hours)]

# Create Polars DataFrame
df = pl.DataFrame(X, schema=[f"feature_{i}" for i in range(n_features)])
df = df.with_columns(pl.Series("date", timestamps))

In [3]:
PATH = Path("./my-test-timelake")

lake = TimeLake.create(
    path=PATH,
    df=df,
    timestamp_column="date",
)
df = lake.read()
df.shape

(720, 103)

In [4]:
lake.read(start_date="2024-01-01", end_date="2024-01-03").shape

(72, 103)

In [5]:
df_update = df[0:10]
lake.upsert(df_update)

In [6]:
df = lake.read()
df.shape

(720, 103)

In [7]:
df

feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,…,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,date,date_day,inserted_at
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[μs],str,datetime[μs]
-1.297589,1.341438,0.1607,1.589851,1.074973,-0.897782,-1.536988,-1.393702,1.142122,2.997212,0.050685,-0.037894,-0.862635,-0.009117,0.214414,-1.297981,0.698573,0.482677,-0.065182,-1.492522,-2.358979,-2.386586,0.154762,0.108464,0.293042,-0.658336,1.684097,0.003296,0.932277,0.397827,-2.255018,0.034284,0.696755,0.974833,-1.028424,-0.816856,-0.131405,…,0.375949,1.344189,-1.837766,-0.627502,0.020443,-0.404291,-1.247424,-0.474928,-0.832249,-1.001298,-0.191364,-0.012009,-1.261858,-0.224284,-0.669871,1.374583,1.019069,1.611863,0.719077,-0.219786,-0.943485,-0.406114,0.15366,0.911051,-2.36134,-0.048301,0.114728,-0.459776,2.070141,0.937419,1.706197,0.208816,-0.458415,2.308985,2024-01-23 19:00:00,"""2024-01-23""",2025-04-18 16:58:15.916209
-2.129139,1.935038,2.322282,-0.377115,-1.279613,0.971864,-0.614757,-1.121818,-0.180111,-0.798085,-0.182268,0.3542,-0.921215,0.326506,0.917333,0.390725,-0.631687,-0.58698,-0.779992,-0.423293,0.450339,0.970624,-0.358076,-0.115218,-1.622717,-1.188441,0.63563,0.874031,-0.7141,-0.007643,0.981492,0.908908,-0.718463,0.799096,-1.555272,-0.958692,-0.684878,…,-0.444942,-0.294616,0.720411,0.835027,-0.079057,1.198712,0.150934,1.190738,-0.326113,-0.579923,0.236742,-0.164517,0.53043,-0.221447,1.269526,-0.571591,1.09687,0.725793,-1.20609,0.508765,1.308781,0.555181,-2.198649,-0.163664,0.589188,1.455106,0.7689,0.230825,0.199155,0.604891,-0.219603,-1.167786,0.365303,0.58189,2024-01-23 11:00:00,"""2024-01-23""",2025-04-18 16:58:15.916209
0.683255,0.793237,-0.776319,1.327152,0.347702,0.536832,0.322011,-0.508964,-1.610528,2.011401,0.357038,-2.839461,-0.069069,0.997772,-1.476616,-0.343481,0.531471,1.589523,-0.639883,1.417986,-1.820622,-1.77105,-1.5093,-0.441324,-0.169539,1.645082,1.120794,-1.156134,0.74349,-0.746493,-1.048664,0.127409,-0.981042,-1.315293,-1.051362,1.952467,-0.237626,…,-0.533936,-0.050547,-1.74767,-0.879362,-0.063292,0.988073,0.083991,0.230089,-1.049961,-1.326237,-0.22988,-1.709795,-0.813049,1.55863,-1.878786,-0.197055,0.185698,0.361221,-0.473822,1.407379,-1.350343,0.15642,-0.357487,-0.238595,-1.972249,-0.57491,1.166273,-0.307955,0.550604,-1.288824,-1.602448,-0.717623,-1.697506,-2.024055,2024-01-23 07:00:00,"""2024-01-23""",2025-04-18 16:58:16.061606
-1.399907,2.046793,1.581106,0.17697,-0.377587,-1.300864,-2.297978,-1.017201,-0.311159,-1.338836,0.889739,-0.902917,-1.405005,0.243537,1.640736,2.79802,-1.47209,-0.470502,-0.563319,-0.332816,-0.687885,-1.039461,0.713166,0.019368,3.070139,0.154352,-0.848424,1.34394,-1.051035,-0.627639,-1.452377,-0.005128,0.582294,0.380855,-0.577819,0.067234,-0.918754,…,0.68952,-0.177691,1.19874,-0.337718,-1.959388,-1.221049,0.593334,-0.704518,-1.224245,0.589644,-1.146239,-0.37854,0.77918,-0.500311,-1.377694,-0.578947,1.549775,-2.154817,-0.096188,-0.08777,0.269174,-0.955974,1.150728,-0.306804,0.509158,-0.873211,-0.5685,-0.210253,-0.856409,0.181695,0.989048,-1.121192,-0.582412,0.231341,2024-

# We try to create a timelake on S3


In [12]:
import dotenv

dotenv.load_dotenv()
lake = TimeLake.create(
    path="s3://my-time-lake/test",
    df=df,
    timestamp_column="date",
    storage_type=StorageType.S3,
)

In [13]:
lake.read().shape

(720, 103)

In [17]:
lake.read(start_date="2024-01-01", end_date="2024-01-01").shape

(24, 103)